### RAG Pipelines - from Data ingestion to Vector DB Pipeline

In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

/Users/amirmansouri/Dev/rag-learn/rag-1/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
### first we need to read all pdfs inside the directory

def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")


Found 3 PDF files to process

Processing: 3681780.3697252.pdf
  ✓ Loaded 8 pages

Processing: Comprehensive Survey of RAG.pdf
  ✓ Loaded 18 pages

Processing: 2411.06037v3.pdf
  ✓ Loaded 25 pages

Total documents loaded: 51


In [5]:
all_pdf_documents

[Document(metadata={'producer': 'iText 4.2.0 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2026-01-28T22:38:28-08:00', 'moddate': '2026-01-28T22:38:28-08:00', 'source': '../data/pdf/3681780.3697252.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': '3681780.3697252.pdf', 'file_type': 'pdf'}, page_content=". \n. \nLatest updates: h\ue03cps://dl.acm.org/doi/10.1145/3681780.3697252\n. \n. \nRESEARCH-ARTICLE\nAutomating Bibliometric Analysis with Sentence Transformers and\nRetrieval-Augmented Generation (RAG): A Pilot Study in Semantic and\nContextual Search for Customized Literature Characterization for High-\nImpact Urban Research\nHAOWEN XU, Oak Ridge National Laboratory, Oak Ridge, TN, United States\n. \nXUEPING LI, The University of Tennessee, Knoxville, Knoxville, TN, United States\n. \nJOSE TUPAYACHI, The University of Tennessee, Knoxville, Knoxville, TN, United States\n. \nJIANMING (JAMIE) LIAN, Oak Ridge National Laboratory, Oak Ridge, TN, United States\n. \nOL

In [7]:
### Text splitting and getting them into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [8]:
chunks = split_documents(all_pdf_documents)
chunks

Split 51 documents into 230 chunks

Example chunk:
Content: . 
. 
Latest updates: hps://dl.acm.org/doi/10.1145/3681780.3697252
. 
. 
RESEARCH-ARTICLE
Automating Bibliometric Analysis with Sentence Transformers and
Retrieval-Augmented Generation (RAG): A Pilot...
Metadata: {'producer': 'iText 4.2.0 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2026-01-28T22:38:28-08:00', 'moddate': '2026-01-28T22:38:28-08:00', 'source': '../data/pdf/3681780.3697252.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': '3681780.3697252.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'iText 4.2.0 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2026-01-28T22:38:28-08:00', 'moddate': '2026-01-28T22:38:28-08:00', 'source': '../data/pdf/3681780.3697252.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': '3681780.3697252.pdf', 'file_type': 'pdf'}, page_content='. \n. \nLatest updates: h\ue03cps://dl.acm.org/doi/10.1145/3681780.3697252\n. \n. \nRESEARCH-ARTICLE\nAutomating Bibliometric Analysis with Sentence Transformers and\nRetrieval-Augmented Generation (RAG): A Pilot Study in Semantic and\nContextual Search for Customized Literature Characterization for High-\nImpact Urban Research\nHAOWEN XU, Oak Ridge National Laboratory, Oak Ridge, TN, United States\n. \nXUEPING LI, The University of Tennessee, Knoxville, Knoxville, TN, United States\n. \nJOSE TUPAYACHI, The University of Tennessee, Knoxville, Knoxville, TN, United States\n. \nJIANMING (JAMIE) LIAN, Oak Ridge National Laboratory, Oak Ridge, TN, United States\n. \nOL

***Embeding and VectorStoreDB***

In [9]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1352.63it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully. Embedding dimension: 384


### VectoreStore


In [11]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 230


In [12]:
chunks

[Document(metadata={'producer': 'iText 4.2.0 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2026-01-28T22:38:28-08:00', 'moddate': '2026-01-28T22:38:28-08:00', 'source': '../data/pdf/3681780.3697252.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': '3681780.3697252.pdf', 'file_type': 'pdf'}, page_content='. \n. \nLatest updates: h\ue03cps://dl.acm.org/doi/10.1145/3681780.3697252\n. \n. \nRESEARCH-ARTICLE\nAutomating Bibliometric Analysis with Sentence Transformers and\nRetrieval-Augmented Generation (RAG): A Pilot Study in Semantic and\nContextual Search for Customized Literature Characterization for High-\nImpact Urban Research\nHAOWEN XU, Oak Ridge National Laboratory, Oak Ridge, TN, United States\n. \nXUEPING LI, The University of Tennessee, Knoxville, Knoxville, TN, United States\n. \nJOSE TUPAYACHI, The University of Tennessee, Knoxville, Knoxville, TN, United States\n. \nJIANMING (JAMIE) LIAN, Oak Ridge National Laboratory, Oak Ridge, TN, United States\n. \nOL

In [13]:
### Convert text to embeddings
texts = [doc.page_content for doc in chunks]

## now we generate our embeddings

embeddings = embedding_manager.generate_embeddings(texts)

## store in the Vector Database
vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 230 texts...


Batches: 100%|██████████| 8/8 [00:02<00:00,  3.03it/s]


Generated embeddings with shape: (230, 384)
Adding 230 documents to vector store...
Successfully added 230 documents to vector store
Total documents in collection: 460


### RAG Retriever Pipeline From VectorStore

In [14]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                print("Distances:", distances[:5])

                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [15]:
print("Collection count:", vectorstore.collection.count())

docs = rag_retriever.retrieve(
    query="Retrieval-Augmented Generation combines retrieval with generation",
    top_k=5,
    score_threshold=0.0
)

print("Returned:", len(docs))
if docs:
    print("Top result distance:", docs[0]["distance"])
    print("Top result score:", docs[0]["similarity_score"])


Collection count: 460
Retrieving documents for query: 'Retrieval-Augmented Generation combines retrieval with generation'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.71it/s]

Generated embeddings with shape: (1, 384)
Distances: [0.42710837721824646, 0.42710837721824646, 0.5356273651123047, 0.5356273651123047, 0.543084979057312]
Retrieved 5 documents (after filtering)
Returned: 5
Top result distance: 0.42710837721824646
Top result score: 0.5728916227817535


In [16]:
rag_retriever

In [17]:
rag_retriever.retrieve("foundational concepts")

Retrieving documents for query: 'foundational concepts'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.22it/s]

Generated embeddings with shape: (1, 384)
Distances: [1.4232573509216309, 1.4232573509216309, 1.4698652029037476, 1.4698652029037476, 1.4792640209197998]
Retrieved 0 documents (after filtering)


[]

In [18]:
rag_retriever.retrieve("This paper presents a comprehensive study of Retrieval-Augmented Generation (RAG), tracing its evolution from foundational concepts to the current state of the art. ")

Retrieving documents for query: 'This paper presents a comprehensive study of Retrieval-Augmented Generation (RAG), tracing its evolution from foundational concepts to the current state of the art. '
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.50it/s]

Generated embeddings with shape: (1, 384)
Distances: [0.3334968686103821, 0.3334968686103821, 0.4511657953262329, 0.4511657953262329, 0.6015252470970154]
Retrieved 5 documents (after filtering)


[{'id': 'doc_8569f0d4_44',
  'content': 'A Comprehensive Survey of Retrieval-Augmented Generation (RAG): Evolution, CurrentLandscapeandFutureDirections\nShailjaGupta(CarnegieMellonUniversity, USA)RajeshRanjan(CarnegieMellonUniversity, USA)SuryaNarayanSingh(BITSindri, India)\nAbstract',
  'metadata': {'doc_index': 44,
   'producer': 'Skia/PDF m131 Google Docs Renderer',
   'source': '../data/pdf/Comprehensive Survey of RAG.pdf',
   'total_pages': 18,
   'page_label': '1',
   'title': 'A Comprehensive Review of Retrieval-Augmented Generation (RAG): Key Challenges and Future Directions',
   'content_length': 241,
   'page': 0,
   'creationdate': '',
   'creator': 'PyPDF',
   'file_type': 'pdf',
   'source_file': 'Comprehensive Survey of RAG.pdf'},
  'similarity_score': 0.6665031313896179,
  'distance': 0.3334968686103821,
  'rank': 1},
 {'id': 'doc_a991f1c4_44',
  'content': 'A Comprehensive Survey of Retrieval-Augmented Generation (RAG): Evolution, CurrentLandscapeandFutureDirections\nSh

### Integration Vectordb Context pipeline With LLM output

In [37]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [38]:
answer = rag_simple("what is RAG?", rag_retriever, llm)
print(answer)

Retrieving documents for query: 'what is RAG?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.08s/it]


Generated embeddings with shape: (1, 384)
Distances: [0.7435992360115051, 0.8802915811538696, 0.9652455449104309]
Retrieved 3 documents (after filtering)


BadRequestError: Error code: 400 - {'error': {'message': 'The model `gemma2-9b-it` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

In [21]:
### Simple RAG pipeline with Ollama LLM (LOCAL, FREE)
from langchain_community.chat_models import ChatOllama
import os
from dotenv import load_dotenv

load_dotenv()

### Initialize the Ollama LLM (make sure `ollama serve` is running)
# Pick one:
# - "llama3.1:8b" (better, heavier)
# - "llama3.2:3b" (lighter, faster)
llm = ChatOllama(model="llama3.1:8b", temperature=0.1)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query, retriever, llm, top_k=3):
    ## retrieve the context
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."

    ## generate the answer using Ollama LLM
    prompt = f"""Use the following context to answer the question concisely.
Context:
{context}

Question: {query}

Answer:"""

    response = llm.invoke(prompt)   # ✅ Ollama: pass a string (not [string])
    return response.content




/var/folders/5x/94mftlfx1tsdlgx5d79s3s1r0000gn/T/ipykernel_40546/1413814251.py:12: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.1:8b", temperature=0.1)


In [40]:
# Example
answer = rag_simple("What is Retrieval Augmented Generation?", rag_retriever, llm)
print(answer)


Retrieving documents for query: 'What is Retrieval Augmented Generation?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.58s/it]


Generated embeddings with shape: (1, 384)
Distances: [0.5441585779190063, 0.6871664524078369, 0.7346802353858948]
Retrieved 3 documents (after filtering)
Retrieval-Augmented Generation (RAG) is a hybrid architecture that integrates retrieval and generative mechanisms to generate human-like text grounded in real-world data.


In [22]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output



In [23]:
# Example usage:
result = rag_advanced("DO BENCHMARK DATASETS HAVE HIGH SUFFICIENT CONTEXT?", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'DO BENCHMARK DATASETS HAVE HIGH SUFFICIENT CONTEXT?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.39it/s]


Generated embeddings with shape: (1, 384)
Distances: [0.6075935959815979, 0.6075935959815979, 0.7356294393539429]
Retrieved 3 documents (after filtering)
Answer: No.
Sources: [{'source': '2411.06037v3.pdf', 'page': 4, 'score': 0.3924064040184021, 'preview': 'Therefore, we use it in Section 4 as our main method for analyzing datasets and model responses.\nLater, in Section 5.1, we use FLAMe as a computationally efficient autorater to provide a signal for\nselective generation. Our comparison with TRUE-NLI and Contains GT confirms that classifying\nsufficien...'}, {'source': '2411.06037v3.pdf', 'page': 4, 'score': 0.3924064040184021, 'preview': 'Therefore, we use it in Section 4 as our main method for analyzing datasets and model responses.\nLater, in Section 5.1, we use FLAMe as a computationally efficient autorater to provide a signal for\nselective generation. Our comparison with TRUE-NLI and Contains GT confirms that classifying\nsufficien...'}, {'source': '2411.06037v3.pdf', 'page':